In [2]:
import itertools
import pandas as pd
import re
from log_class import log_recorder

LOG_DIR = "./test_data2/"
report_csv = "report.csv"
LOG_file = "LOG"


# from log_class import log_recorder
COMPACTION_LOG_HEAD = "/compaction/compaction_job.cc:755"
FLUSH_LOG_BEGIN = "flush_started"
FLUSH_LOG_END = "flush_finished"
FLUSH_FILE_CREATEION = "table_file_creation"

def load_log_and_qps(log_file, ground_truth_csv):
    # load the data
    return log_recorder(log_file,ground_truth_csv)

data_set = load_log_and_qps(LOG_DIR+LOG_file, LOG_DIR+report_csv)

# Cut the timeline into pieces according to the time slice size

In [16]:
ms_to_sec  = 1000000
time_slice = 100000 # 100 miusec, 100,000ms
switch_ratio = ms_to_sec/time_slice

real_time_speed=data_set.qps_df
flush_jobs = data_set.flush_df
# op_type, 0 for flush, 1 for compaction; 
# triggered or not, 0 for not triggered, 1 for running
# input size, how many bytes have been read from disk
# total_operation_bytes
# job_id, i don't kown whether it can be used or not.
input_tuple=[0,0,0,12345,7] 

# then we put all the operations into a series of input tupler stack
#        -----
#      ----
# --- -- 
# -- -------
# 1 2 3 4 5 6 7 8 9 10



# create the bucket first

# print(int(flush_jobs[flush_jobs["job"]==11902]["start_time"]/time_slice))


In [31]:

bucket = []
for i in range(int(real_time_speed.tail(1)["microsecs_elapsed"] * switch_ratio)):
    bucket.append([])
# then we use a bucket sort idea to count down the rest things
for flush_job in data_set.flush_df.iloc():
    # indices = (int(flush_job["start_time"]/time_slice), flush_job["end_time"]/time_slice)
    start_index = int(flush_job["start_time"]/time_slice)
    end_index=int(flush_job["end_time"]/time_slice) + 1
    payload = round(flush_job["flush_size"] / (1024*1024) ,2) # change to MB will be easier to calculate
    job_id = flush_job["job"]

    if start_index >= len(bucket)-10 or end_index >= len(bucket)-5: # the tail part is not accurant
        break
    for bucket_element in bucket[start_index:end_index]:
        bucket_element.append([0,0,payload,job_id])


for compaction_job in data_set.compaction_df.iloc():
    start_index = int(compaction_job["start_time"]/time_slice)
    end_index=int(compaction_job["end_time"]/time_slice) + 1
    input_size = round(compaction_job["input_data_size"] / (1024*1024) ,2) # change to MB will be easier to calculate
    output_size = round(compaction_job["total_output_size"] / (1024*1024) ,2)
    job_id = compaction_job["job"]
    if start_index >= len(bucket)-10 or end_index >= len(bucket)-5: # the tail part is not accurant
        break
    compaction_tuple=[1,input_size,output_size,job_id]
    for bucket_element in bucket[start_index:end_index]:
        bucket_element.append(compaction_tuple)

result_tensor = real_time_speed["interval_qps"]

In [30]:
# create the transformer model

import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerModel(nn.Module):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != src.size(0):
            device = src.device
            mask = self._generate_square_subsequent_mask(src.size(0)).to(device)
            self.src_mask = mask

        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask)
        output = self.decoder(output)
        return output
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)